In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
!pip install transformers 
!pip install sentencepiece
import torch
from transformers import TrainingArguments, Trainer
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import plot_confusion_matrix
import numpy as np
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification  
import pandas as pd


     |████████████████████████████████| 2.3MB 6.9MB/s 
     |████████████████████████████████| 901kB 46.8MB/s 
     |████████████████████████████████| 3.3MB 52.4MB/s 
     |████████████████████████████████| 1.2MB 8.9MB/s 


In [3]:
tokenizer = AutoTokenizer.from_pretrained("Hate-speech-CNERG/dehatebert-mono-arabic")
model = AutoModelForSequenceClassification.from_pretrained("Hate-speech-CNERG/dehatebert-mono-arabic")

In [4]:
df_test = pd.read_excel('/content/drive/MyDrive/MasterThesis/DataSet-forUsing/semeval2020/test.xlsx')
df_test_labels = pd.read_excel('/content/drive/MyDrive/MasterThesis/DataSet-forUsing/semeval2020/offLabel.xlsx')
# df_dev = pd.read_excel('/content/drive/MyDrive/MasterThesis/DataSet-forUsing/semeval2020/dev.xlsx')
df_test_labels.head

<bound method NDFrame.head of        labels
0         OFF
1         OFF
2     NOT_OFF
3         OFF
4     NOT_OFF
...       ...
1995      OFF
1996  NOT_OFF
1997  NOT_OFF
1998      OFF
1999      OFF

[2000 rows x 1 columns]>

In [5]:
#Label Encoding to convert labels to integer values

y_test=df_test_labels.labels
print(y_test[0:20])
LE = LabelEncoder()
y_test=LE.fit_transform(y_test)
print(y_test[0:20])
y_test.shape

0         OFF
1         OFF
2     NOT_OFF
3         OFF
4     NOT_OFF
5     NOT_OFF
6     NOT_OFF
7     NOT_OFF
8     NOT_OFF
9     NOT_OFF
10        OFF
11    NOT_OFF
12    NOT_OFF
13    NOT_OFF
14    NOT_OFF
15    NOT_OFF
16        OFF
17    NOT_OFF
18        OFF
19    NOT_OFF
Name: labels, dtype: object
[1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0]


(2000,)

In [6]:
%load_ext google.colab.data_table
df_test
# grouped = df_dev.groupby(df_dev.hs)
# y_test_HS = grouped.get_group("HS")
# y_test_Not_HS = grouped.get_group("NOT_HS")
# y_test_HS=y_test_HS.label.values
# y_test_Not_HS=y_test_Not_HS.label.values


,tweets
0,@USER اما انت تقعد طول عمرك لا مبدا ولا راي ث...
1,@USER @USER بتخاف نسوانك يزعلوا ولا ايه 😂 اه ...
2,RT @USER: يا عـسانـى نـبـقى يا عـمري حـبايـب ...
3,RT @USER: باقي البيان وينو ما شفنه يا برهان <...
4,@USER @USER اللهم انت الشافي المعافي اشفيه وج...
...,...
1995,RT @USER: الله لايوفقك يا مهند عسيري يا معوق ...
1996,RT @USER: @USER حبيبي يا يوسف وانت طيب يا صاح...
1997,RT @USER: يا بو محمد عشت يا طيب الفال<LF>عاشت...
1998,أنا مستني الحلقة بقالي سنتين يا بضان يا ابن ا...


In [7]:
classes = ["NOT_OFF", "OFF"]

In [9]:
# preprocessing function
import re
text_cleaning_re = "[a-zA-Z]|\d+|[٠١٢٣٤٥٦٧٨٩]|[.#،<>@,\\-_”“٪ًَ]"
def preprocess(text):
  # search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!','#']
  # replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ',' ']
  #remove tashkeel
  p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
  text = re.sub(p_tashkeel,"", str(text))
  # text = text.replace('وو', 'و')
  # text = text.replace('يي', 'ي')
  # text = text.replace('اا', 'ا')
  text = re.sub(text_cleaning_re, ' ', str(text).lower()).strip()
  #remove longation
  text = re.sub(r'(.)\1+', r'\1\1', str(text)) 
  text = re.sub("[إأآا]", "ا", str(text))
  text = re.sub("ى", "ي", str(text))
  # text = re.sub("ؤ", "ء", text)
  # text = re.sub("ئ", "ء", text)
  text = re.sub("ة", "ه", str(text))
  # for i in range(0, len(search)):
  #       text = text.replace(search[i], replace[i])
  
   #trim    
  # text = text.strip()

  tokens = []
  for token in text.split():
    # if token not in stop_words:
      tokens.append(token)
  return " ".join(tokens)

x_test = df_test.tweets.apply(lambda x: preprocess(x))


In [10]:
# y_pred = []
pred1 = []
for tweet in x_test:
  paraphrase = tokenizer(tweet, return_tensors="pt")
  paraphrase_classification_logits = model(**paraphrase).logits
  paraphrase_results = torch.softmax(paraphrase_classification_logits, dim=1).tolist()[0]
  # y_pred.append(int(paraphrase_results[0] > paraphrase_results[1]))
  # pred1.append(int(paraphrase_results[0] > 0))
  pred1.append(paraphrase_results)
  y_pred = np.argmax(pred1, axis=1)
  # pred.append(int(round(paraphrase_results[0] * 100)))

In [11]:
y_pred[:20]

array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0])

In [12]:
y_test[:20]

array([1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0])

In [13]:
print("Accuracy", accuracy_score(y_test, y_pred))

print("\nConfusion matrix\n",confusion_matrix(y_test,y_pred))
print("\nClassification report\n",classification_report(y_test,y_pred))

Accuracy 0.6755

Confusion matrix
 [[1095  503]
 [ 146  256]]

Classification report
               precision    recall  f1-score   support

           0       0.88      0.69      0.77      1598
           1       0.34      0.64      0.44       402

    accuracy                           0.68      2000
   macro avg       0.61      0.66      0.61      2000
weighted avg       0.77      0.68      0.70      2000



# Experment 10 sample

In [ ]:
pred1 = []
for tweet in df_test.tweets[:10]:
  paraphrase = tokenizer(tweet, return_tensors="pt")
  paraphrase_classification_logits = model(**paraphrase).logits
  paraphrase_results = torch.softmax(paraphrase_classification_logits, dim=1).tolist()[0]
  pred1.append(int(paraphrase_results[0] > paraphrase_results[1]))
  # pred1=pred1.append(paraphrase_results)
  # y_pred = np.argmax(pred1, axis=1)
  # pred.append(int(round(paraphrase_results[0] * 100)))

In [ ]:
pred1[:10]

In [ ]:
y_test[:10]

In [ ]:
confusion_matrix(y_test[:10],pred1[:10])